In [7]:
import json

with open('notes.json') as f:
    data = json.load(f)

data = data['440']

def get_note(i, j):
    return fmt_note(data[i+j]['note'])

def fmt_note(note):
    return note.replace('#', '_sh_').upper()

bufer = ""
bufer += "/* Useful type aliases */\n"
bufer += "pub type Scale = [f32; 7];\n"
bufer += "pub type Lead = [&'static Scale; 3];\n"
bufer += "pub type Chord = [f32; 3];\n"
bufer += "pub type ChordPool = [&'static Chord; 7];\n"
bufer += "pub struct Key {\n\tpub lead: Lead,\n\tpub chords: ChordPool\n}\n"
bufer += "\n"

# notes
bufer += "/* Single notes */\n"
for element in data:
    bufer += f"pub const {fmt_note(element['note'])}: f32 = {float(element['frequency'])}" + ";\n"
    
bufer += "\n"

# minor scales
bufer += "/* Scales */\n"
minor_scales = {}
stages = [0, 2, 3, 5, 7, 8, 10]
for i in range(0, (len(data)-24)):
    minor_scale = [get_note(i, j) for j in stages]
    minor_scales[get_note(i, 0) + 'M'] = minor_scale
    bufer += f"pub static {get_note(i, 0)}_MINOR_SCALE: Scale = [{', '.join(minor_scale)}];\n"

bufer += "\n"

# major scales
major_scales = {}
stages = [0, 2, 4, 5, 7, 9, 11]
for i in range(0, (len(data)-24)):
    major_scale = [get_note(i, j) for j in stages]
    major_scales[get_note(i, 0)] = major_scale
    bufer += f"pub static {get_note(i, 0)}_MAJOR_SCALE: Scale = [{', '.join(major_scale)}];\n"

bufer += "\n"

# minor leads
bufer += "/* Leads */\n"
minor_leads = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
stages = [4, 5, 6]
for el in minor_leads:
    lead = ", ".join([f"&{el}{s}_MINOR_SCALE" for s in stages])
    bufer += f"pub static {el}M_LEAD: Lead = [{lead}];\n"
    
bufer += "\n"

# major leads
major_leads = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
stages = [4, 5, 6]
for el in major_leads:
    lead = ", ".join([f"&{el}{s}_MAJOR_SCALE" for s in stages])
    if len(el) > 1:
        el = el[:-1]
    bufer += f"pub static {el}_LEAD: Lead = [{lead}];\n"
    
bufer += "\n"


# minor chords
bufer += "/* Chords */\n"
stages = [0, 3, 5]
for i in range(12 * 2, (len(data) - 3 * 12)):
    minor_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_MINOR_CHORD: Chord = [{minor_chord}];\n"

bufer += "\n"

# major chords
stages = [0, 4, 5]
for i in range(12 * 2, (len(data) - 3 * 12)):
    major_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_MAJOR_CHORD: Chord = [{major_chord}];\n"
    
bufer += "\n"

# deminished chords
stages = [0, 3, 6]
for i in range(12 * 2, (len(data) - 3 * 12)):
    dem_chord = ", ".join([get_note(i, j) for j in stages])
    bufer += f"pub static {get_note(i, 0)}_DEM_CHORD: Chord = [{dem_chord}];\n"
    
bufer += "\n"

# minor chords pools
bufer += "/* Chord pools */\n"
note_lits = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
octaves = ["3"]
minor_stages = [0, 3, 4]
major_stages = [2, 5, 6]
dem_stages = [1]
for b in note_lits:
    minor_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_MINOR_CHORD" for j in minor_stages for o in octaves])
    major_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_MAJOR_CHORD" for j in major_stages for o in octaves])
    dem_chords = ", ".join([f"&{minor_scales[b + o + 'M'][j]}_DEM_CHORD" for j in dem_stages for o in octaves])
    bufer += f"pub static {b}M_CHORD_POOL: ChordPool = [{minor_chords}, {major_chords}, {dem_chords}];\n"
    
# major chords pools
note_lits = ["A", "A_SH_", "B", "C", "C_SH_", "D", "D_SH_", "E", "F", "F_SH_", "G", "G_SH_"]
octaves = ["3"]
minor_stages = [1, 2, 5]
major_stages = [0, 3, 4]
dem_stages = [6]
for b in note_lits:
    minor_chords = ", ".join([f"&{major_scales[b + o][j]}_MINOR_CHORD" for j in minor_stages for o in octaves])
    major_chords = ", ".join([f"&{major_scales[b + o][j]}_MAJOR_CHORD" for j in major_stages for o in octaves])
    dem_chords = ", ".join([f"&{major_scales[b + o][j]}_DEM_CHORD" for j in dem_stages for o in octaves])
    if len(b) > 1:
        b = b[:-1]
    bufer += f"pub static {b}_CHORD_POOL: ChordPool = [{minor_chords}, {major_chords}, {dem_chords}];\n"
           
bufer += "\n"

bufer += "/* Keys */\n"
for note in note_lits:
    bufer += f"pub static {note}M: Key = Key {{ lead: {note}M_LEAD, chords: {note}M_CHORD_POOL }};\n"
    if len(note) > 1:
        note = note[:-1]
    bufer += f"pub static {note}: Key = Key {{ lead: {note}_LEAD, chords: {note}_CHORD_POOL }};\n"

bufer += "\n"
keys = ", ".join(f"&{note}M" for note in note_lits) + ", " + ", ".join(f"&{note}" for note in note_lits if len(note) == 1) + ", " + ", ".join(f"&{note[:-1]}" for note in note_lits if len(note) > 1)
bufer += f"pub static KEYS: [&Key; 24] = [{keys}];\n"
    
with open('notes_data.rs', 'w') as d:
    d.write(bufer)